In [1]:
import os , sys ,re ,time ,nltk, json
import xml.etree.cElementTree as et
from collections import defaultdict

In [2]:
by_article_small = r'D:\3rd SEM\IRE\Major Project\Datasets\ground-truth-training-byarticle-20181122.xml'
by_article_big = r'D:\3rd SEM\IRE\Major Project\Datasets\articles-training-byarticle-20181122.xml'

In [3]:
dd = defaultdict(lambda : defaultdict(lambda : "Nothing here")) # dd[id] = [[bias],[hyperpartitian],[url],[labeled-by]]

In [6]:
def strip_tag_name(t):
    # t = elem.tag
    idx = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

final_list =[]
def parser(dump):
    obj = et.iterparse(dump, events = ('start' ,'end'))
    obj = iter(obj)
    pp=[]
    
    for event, elem in obj :
        tag = strip_tag_name(elem.tag)
        if event == 'end':
            if tag == 'article' :
                idd = int(elem.attrib['id'])
                url = elem.attrib['url']
                hyperpartisan = elem.attrib['hyperpartisan']
#                 bias = elem.attrib['bias']
                labeled_by = elem.attrib['labeled-by']
                
                dd[idd] = { 'hyperpartisan': hyperpartisan ,
#                            'bias' : bias,
                           'url' : url,
                           'labeled-by' :labeled_by
                            }


In [7]:
parser(by_article_small)

In [ ]:
big_xml_path2 =  r'D:\3rd SEM\IRE\Major Project\Datasets\articles-validation-bypublisher-20181122\test2.xml'

In [8]:
dd

defaultdict(<function __main__.<lambda>()>,
            {0: {'hyperpartisan': 'true',
              'url': 'https://www.opednews.com/articles/Kucinich-Reclaiming-the-m-by-Dennis-Kucinich-Banks_Debt_Funding_Money-170910-112.html',
              'labeled-by': 'article'},
             1: {'hyperpartisan': 'true',
              'url': 'http://bipartisanreport.com/2017/10/12/trump-just-woke-up-viciously-attacked-puerto-ricans-on-twitter-like-a-cruel-old-man/',
              'labeled-by': 'article'},
             2: {'hyperpartisan': 'true',
              'url': 'https://www.reviewjournal.com/opinion/letters/liberals-wailing-about-gun-control-but-what-about-abortion/',
              'labeled-by': 'article'},
             3: {'hyperpartisan': 'true',
              'url': 'https://www.redcuprebellion.com/2017/9/24/16358776/laremy-tunsil-national-anthem-kneeling-protest-donald-trump',
              'labeled-by': 'article'},
             4: {'hyperpartisan': 'false',
              'url': 'https:

In [9]:
def process_text(text):
    tokens = re.split(r"[^A-Za-z0-9]+", text)
    tokens = [w.casefold() for w in tokens]
#     tokens = [x for x in tokens if len(x) >= 3 and  len(x) <= 20 ]
#     tokens = [x for x in tokens if len(x) >= 3 and x not in stop_words and  len(x) <= 20 ]
    
#     tokens = [ dpp(x) for x in tokens]
    z =  ' '.join(tokens)
    return z

In [10]:
def parser2(dump):
    obj = et.iterparse(dump, events = ('start' ,'end'))
    obj = iter(obj)
    pp =[]
    for event, elem in obj :
        tag = strip_tag_name(elem.tag)
        if event == 'end':
            if tag == 'p':
                if elem.text is not None :
                    pp.append(elem.text)
                
        
            if tag == 'article' :
                idd = int(elem.attrib['id'])
                try:
                    published_at = elem.attrib['published-at']
                except :
                    published_at ='unknown'
                title = elem.attrib['title']
                
                z = ' '.join(pp)
                x = process_text(z)
            
                dd[idd].update({ 'published-at': published_at ,
                           'title' : title,
                           'text' : x,
                            })
                pp.clear()
            elem.clear()
    
        
        
        

In [11]:
parser2(by_article_big)

In [12]:
cc = 0
for k,v in dd.items():
    cc += 1
    if cc == 10 : break
    print(k , "==>")
    for a,b in v.items():
        print(a, " : " ,b)

0 ==>
hyperpartisan  :  true
url  :  https://www.opednews.com/articles/Kucinich-Reclaiming-the-m-by-Dennis-Kucinich-Banks_Debt_Funding_Money-170910-112.html
labeled-by  :  article
published-at  :  2017-09-10
title  :  Kucinich: Reclaiming the money power
text  :  money the walls are closing in on congress terrifying walls of water from hurricanes harvey and irma which when the damage is totaled could rise to a half trillion dollars the walls of war the multi trillion dollar ongoing cost of afghanistan iraq and other interventions the crumbling walls of the u s infrastructure which need at least 3 trillion to be repaired or replaced a wall of 11 million undocumented immigrants whose deportation could easily cost 200 billion the planned wall at the mexican border which some estimates place at 67 billion then there is the wall of all the 20 trillion national debt the walls of debt are closing in at moments of crisis in our nation in addition to invoking the assistance of higher powers we 

In [13]:
with open('data_final_byarticle.json', 'w') as fp:
    json.dump(dd, fp,indent=2)